In [29]:
@generated function harmonic(values::Vararg{Number})
    size = length(values)
    exp=:(1/values[1])
    for i= 2:size
        exp= :($exp+(1/values[$i]))
    end
    return :($size/$exp)
end

harmonic (generic function with 1 method)

In [30]:
harmonic(2,2,5,7)

2.9787234042553195

In [64]:
function autodiff(ex::Expr)::Expr
    print("a ",ex.args,"\n")
    if length(ex.args)<=3
        return autodiff(ex.args[1],ex.args[2:end])
    else
        return autodiff(Expr(:call,ex.args[1],Expr(:call,ex.args[1],ex.args[2:end-1]...),ex.args[end]))
    end
end
function autodiff(ex::Number)::Int64
    print("c ",ex,"\n")
    return 0
end
function autodiff(ex::Symbol)::Int64
    print("d ",ex,"\n")
    return :(1)
end
function autodiff(op::Symbol,args)::Expr
    print("b ",args,"\n")
    if op==:*
        return Expr(:call,:+,Expr(:call,:*,autodiff(args[1]),args[2]),Expr(:call,:*,args[1],autodiff(args[2])))
    elseif op==:+
        return Expr(:call,:+,autodiff(args[1]),autodiff(args[2]))
    elseif op==:-
        return Expr(:call,:-,autodiff(args[1]),autodiff(args[2]))
    elseif op==:/
        nomin=Expr(:call,:-,Expr(:call,:*,autodiff(args[1]),args[2]),Expr(:call,:*,args[1],autodiff(args[2])))
        denom=Expr(:call,:^,args[2],2)
        return Expr(:call,:/,nomin,denom)
    end
    return :()
end

autodiff (generic function with 4 methods)

In [65]:
deriv=autodiff(:((5*x*x+4*x)/(3*x*x*x+2*x*x+x)));

a Any[:/, :(5 * x * x + 4x), :(3 * x * x * x + 2 * x * x + x)]
b Any[:(5 * x * x + 4x), :(3 * x * x * x + 2 * x * x + x)]
a Any[:+, :(5 * x * x), :(4x)]
b Any[:(5 * x * x), :(4x)]
a Any[:*, 5, :x, :x]
a Any[:*, :(5x), :x]
b Any[:(5x), :x]
a Any[:*, 5, :x]
b Any[5, :x]
c 5
d x
d x
a Any[:*, 4, :x]
b Any[4, :x]
c 4
d x
a Any[:+, :(3 * x * x * x), :(2 * x * x), :x]
a Any[:+, :(3 * x * x * x + 2 * x * x), :x]
b Any[:(3 * x * x * x + 2 * x * x), :x]
a Any[:+, :(3 * x * x * x), :(2 * x * x)]
b Any[:(3 * x * x * x), :(2 * x * x)]
a Any[:*, 3, :x, :x, :x]
a Any[:*, :(3 * x * x), :x]
b Any[:(3 * x * x), :x]
a Any[:*, 3, :x, :x]
a Any[:*, :(3x), :x]
b Any[:(3x), :x]
a Any[:*, 3, :x]
b Any[3, :x]
c 3
d x
d x
d x
a Any[:*, 2, :x, :x]
a Any[:*, :(2x), :x]
b Any[:(2x), :x]
a Any[:*, 2, :x]
b Any[2, :x]
c 2
d x
d x
d x


In [66]:
deriv

:(((((0x + 5 * 1) * x + (5x) * 1) + (0x + 4 * 1)) * (3 * x * x * x + 2 * x * x + x) - (5 * x * x + 4x) * (((((0x + 3 * 1) * x + (3x) * 1) * x + (3 * x * x) * 1) + ((0x + 2 * 1) * x + (2x) * 1)) + 1)) / (3 * x * x * x + 2 * x * x + x) ^ 2)

In [67]:
@eval f(x)=$deriv
f(5)
# Zgadza się z wolfram alpha ❤ http://www.wolframalpha.com/input/?i=evaluate+(((5*x*x%2B4*x)%2F(3*x*x*x%2B2*x*x%2Bx))%27)+at+x%3D5

-0.06733369388858843

In [2]:
f(z)=5*z*2+10*z

f (generic function with 1 method)

In [69]:
deriv_from_fun=parse(replace(replace(string(code_lowered(f)[1].code[end]),"_2","x"),"return",""))
autodiff(deriv_from_fun)

a Any[:+, :(5 * x * 2), :(10x)]
b Any[:(5 * x * 2), :(10x)]
a Any[:*, 5, :x, 2]
a Any[:*, :(5x), 2]
b Any[:(5x), 2]
a Any[:*, 5, :x]
b Any[5, :x]
c 5
d x
c 2
a Any[:*, 10, :x]
b Any[10, :x]
c 10
d x


:(((0x + 5 * 1) * 2 + (5x) * 0) + (0x + 10 * 1))

In [72]:
autodiff(:(5x))

a Any[:*, 5, :x]
b Any[5, :x]
c 5
d x


:(0x + 5 * 1)